In [1]:
%pip install -q opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\aditya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import cv2
import os

video_path = "../sample_data/stempade_1.mp4"
output_dir = "extracted_frames"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps

print(f"📼 FPS: {fps}, Total Frames: {frame_count}, Duration: {duration:.2f} seconds")

# Extract 1 frame per second
current_second = 0
frame_id = 0

while True:
    # Go to the frame at the given second
    frame_number = int(current_second * fps)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()

    if not ret:
        break

    output_path = os.path.join(output_dir, f"frame_{current_second:02d}.jpg")
    cv2.imwrite(output_path, frame)
    print(f"✅ Saved frame at {current_second}s -> {output_path}")

    current_second += 1
    if current_second > duration:
        break

cap.release()


📼 FPS: 23.976, Total Frames: 97, Duration: 4.05 seconds
✅ Saved frame at 0s -> extracted_frames\frame_00.jpg
✅ Saved frame at 1s -> extracted_frames\frame_01.jpg
✅ Saved frame at 2s -> extracted_frames\frame_02.jpg
✅ Saved frame at 3s -> extracted_frames\frame_03.jpg
✅ Saved frame at 4s -> extracted_frames\frame_04.jpg


In [5]:
import os
import requests

API_URL = "https://xln0ug8mnka0z8-8000.proxy.runpod.net/ask_image"
FRAME_DIR = "extracted_frames"
PROMPT = "Do you see any signs of a stampede or crowd panic? Reply 'Yes' or 'No' only."

for filename in sorted(os.listdir(FRAME_DIR)):
    if filename.endswith(".jpg"):
        frame_path = os.path.join(FRAME_DIR, filename)
        with open(frame_path, "rb") as f:
            files = {"image": (filename, f, "image/jpeg")}
            data = {"prompt": PROMPT}
            print(f"📤 Sending {filename}...")
            try:
                response = requests.post(API_URL, files=files, data=data)
                if response.ok:
                    result = response.json()
                    print(f"🧠 Response for {filename}: {result['text']}")
                else:
                    print(f"❌ Failed on {filename}: {response.status_code} - {response.text}")
            except Exception as e:
                print(f"⚠️ Error for {filename}: {e}")


📤 Sending frame_00.jpg...
🧠 Response for frame_00.jpg: Yes
📤 Sending frame_01.jpg...
🧠 Response for frame_01.jpg: Yes
📤 Sending frame_02.jpg...
🧠 Response for frame_02.jpg: Yes
📤 Sending frame_03.jpg...
🧠 Response for frame_03.jpg: Yes
📤 Sending frame_04.jpg...
🧠 Response for frame_04.jpg: Yes
